# Notebook 01: The Quantum Eraser — Explained and Debunked

**Reference**: Kim, Yu, Kulik, Shih, Scully, PRL 84, 1-5 (2000)

## The Question

The delayed-choice quantum eraser experiment seems to show that a **future** choice (whether to erase which-path information) affects a **past** measurement result. Does this mean we can send messages backward in time?

## The Answer

**No.** The quantum eraser does NOT demonstrate backward-in-time signaling. The key insight:

- The total pattern at detector D0 is **always featureless** — no interference, regardless of what happens to the idler photon.
- Interference fringes only appear when you **post-select** (look at D0 events coincident with specific idler detectors).
- Post-selection requires classical information from the idler side, which travels at the speed of light or slower.

Let's verify this computationally.

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt

from src.eraser.kim_eraser import KimQuantumEraser
from src.analysis.statistics import fringe_visibility

## 1. The Kim et al. Experimental Setup

The experiment uses **SPDC** (spontaneous parametric down-conversion) to produce entangled photon pairs:

```
                                 ┌─────── D1 (erased: fringes)
                                 │
  Signal → [Double Slit] → D0   BS network → D2 (erased: anti-fringes)
                                 │
  Source (SPDC)                  ├─────── D3 (which-path: no fringes)
                                 │
  Idler ─────────────────────────┴─────── D4 (which-path: no fringes)
```

- **D1, D2**: Which-path info is erased (beam splitter combines paths)
- **D3, D4**: Which-path info is preserved (each receives from only one slit)

The idler photon can be detected **after** the signal photon hits D0.

In [ ]:
# Run the quantum eraser simulation
eraser = KimQuantumEraser(n_experiments=50000)
result = eraser.run_experiment()

print(f"Total photon pairs simulated: {result.n_experiments}")
counts = result.counter.detector_counts()
for det, n in sorted(counts.items()):
    print(f"  {det}: {n} events ({100*n/result.n_experiments:.1f}%)")

## 2. The Total D0 Pattern — Always Featureless

This is the **critical point**: if you just look at all D0 detections (without knowing which idler detector fired), you see **no interference pattern**. This means Alice at D0 **cannot tell** what happens to the idler photon.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 9))

# Total D0 pattern
centers, total = result.total_d0_pattern(n_bins=80)
axes[0, 0].bar(centers * 1e3, total, width=(centers[1]-centers[0])*1e3, 
               color='gray', alpha=0.7)
axes[0, 0].set_title(f'Total D0 (ALL events)\nVisibility = {fringe_visibility(total):.4f}', fontsize=11)
axes[0, 0].set_xlabel('Position (mm)')
axes[0, 0].set_ylabel('Counts')

# D1 coincidences (erased → fringes)
c, d1 = result.coincidence_pattern('D1', n_bins=80)
axes[0, 1].bar(c * 1e3, d1, width=(c[1]-c[0])*1e3, color='blue', alpha=0.7)
vis_d1 = fringe_visibility(d1) if d1.sum() > 100 else 0
axes[0, 1].set_title(f'D0 ∩ D1 (erased)\nVisibility = {vis_d1:.4f}', fontsize=11)
axes[0, 1].set_xlabel('Position (mm)')

# D2 coincidences (erased → anti-fringes)
c, d2 = result.coincidence_pattern('D2', n_bins=80)
axes[0, 2].bar(c * 1e3, d2, width=(c[1]-c[0])*1e3, color='red', alpha=0.7)
vis_d2 = fringe_visibility(d2) if d2.sum() > 100 else 0
axes[0, 2].set_title(f'D0 ∩ D2 (erased, anti-phase)\nVisibility = {vis_d2:.4f}', fontsize=11)
axes[0, 2].set_xlabel('Position (mm)')

# D3 coincidences (which-path → no fringes)
c, d3 = result.coincidence_pattern('D3', n_bins=80)
axes[1, 0].bar(c * 1e3, d3, width=(c[1]-c[0])*1e3, color='green', alpha=0.7)
vis_d3 = fringe_visibility(d3) if d3.sum() > 100 else 0
axes[1, 0].set_title(f'D0 ∩ D3 (which-path: upper)\nVisibility = {vis_d3:.4f}', fontsize=11)
axes[1, 0].set_xlabel('Position (mm)')
axes[1, 0].set_ylabel('Counts')

# D4 coincidences (which-path → no fringes)
c, d4 = result.coincidence_pattern('D4', n_bins=80)
axes[1, 1].bar(c * 1e3, d4, width=(c[1]-c[0])*1e3, color='orange', alpha=0.7)
vis_d4 = fringe_visibility(d4) if d4.sum() > 100 else 0
axes[1, 1].set_title(f'D0 ∩ D4 (which-path: lower)\nVisibility = {vis_d4:.4f}', fontsize=11)
axes[1, 1].set_xlabel('Position (mm)')

# D1 + D2 combined (fringes cancel!)
axes[1, 2].bar(centers * 1e3, d1 + d2, width=(centers[1]-centers[0])*1e3, 
               color='purple', alpha=0.7)
vis_combined = fringe_visibility(d1 + d2) if (d1+d2).sum() > 100 else 0
axes[1, 2].set_title(f'D1 + D2 combined\nVisibility = {vis_combined:.4f}\n(fringes cancel!)', fontsize=11)
axes[1, 2].set_xlabel('Position (mm)')

plt.suptitle('Kim et al. Quantum Eraser: Why It Is NOT Time Travel', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 3. The Fringe Cancellation Mechanism

**Why** is the total D0 pattern featureless? Because:

- D1 shows **fringes**: $P(x|D1) \propto |A_{upper}(x) + A_{lower}(x)|^2$
- D2 shows **anti-fringes**: $P(x|D2) \propto |A_{upper}(x) - A_{lower}(x)|^2$
- D1 + D2 = $|A_u + A_l|^2 + |A_u - A_l|^2 = 2(|A_u|^2 + |A_l|^2)$ → **no fringes**

The complementary fringes of D1 and D2 **exactly cancel** when combined. This cancellation is guaranteed by the no-signaling theorem.

In [ ]:
# Demonstrate fringe cancellation analytically
x = np.linspace(-0.004, 0.004, 500)
d = eraser.d  # slit separation
lam = eraser.lam  # wavelength
L = eraser.L  # screen distance

# Amplitudes from each slit
envelope = np.sinc(eraser.a * x / (lam * L))
phase = np.pi * d * x / (lam * L)
A_upper = envelope * np.exp(1j * phase)
A_lower = envelope * np.exp(-1j * phase)

# Patterns
P_D1 = np.abs(A_upper + A_lower)**2   # fringes
P_D2 = np.abs(A_upper - A_lower)**2   # anti-fringes
P_total = P_D1 + P_D2                  # cancels!
P_single = np.abs(A_upper)**2          # single slit (D3 or D4)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(x * 1e3, P_D1 / P_D1.max(), 'b-', label='D1 (fringes)', alpha=0.7)
ax.plot(x * 1e3, P_D2 / P_D2.max(), 'r--', label='D2 (anti-fringes)', alpha=0.7)
ax.plot(x * 1e3, P_total / P_total.max(), 'k-', linewidth=2, label='D1+D2 (CANCELS)', alpha=0.9)
ax.plot(x * 1e3, 2 * P_single / P_total.max(), 'g:', linewidth=2, label='2 × single slit', alpha=0.7)
ax.set_xlabel('Position on D0 screen (mm)', fontsize=12)
ax.set_ylabel('Normalized Intensity', fontsize=12)
ax.set_title('Fringe Cancellation: D1 + D2 = featureless envelope', fontsize=13)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Wheeler's Delayed Choice Experiment

A simpler version of the delayed-choice idea: a Mach-Zehnder interferometer where the second beam splitter (BS2) is inserted or removed **after** the photon enters.

- **With BS2**: interference (wave behavior)
- **Without BS2**: which-path (particle behavior)

Again, the photon seems to "know in advance" — but no information flows backward.

In [ ]:
from src.eraser.kim_eraser import WheelerDelayedChoice

wheeler = WheelerDelayedChoice(n_experiments=5000)

# Phase sweep with and without BS2
result_with_bs2 = wheeler.phase_sweep(n_phases=50, insert_bs2=True)
result_without_bs2 = wheeler.phase_sweep(n_phases=50, insert_bs2=False)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(result_with_bs2['phases'], result_with_bs2['p0'], 'b.-', label='Detector 0')
ax1.plot(result_with_bs2['phases'], result_with_bs2['p1'], 'r.-', label='Detector 1')
ax1.set_xlabel('Phase (rad)')
ax1.set_ylabel('Detection Probability')
ax1.set_title('BS2 Present: Interference (wave)', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(result_without_bs2['phases'], result_without_bs2['p0'], 'b.-', label='Detector 0')
ax2.plot(result_without_bs2['phases'], result_without_bs2['p1'], 'r.-', label='Detector 1')
ax2.set_xlabel('Phase (rad)')
ax2.set_ylabel('Detection Probability')
ax2.set_title('BS2 Absent: No Interference (particle)', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.suptitle('Wheeler Delayed-Choice Experiment', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Key Takeaways

1. **The total D0 pattern is always featureless** — no interference fringes are visible without coincidence counting.

2. **Fringes only appear in post-selected subsets** — and these subsets can only be identified using classical information from the idler detectors.

3. **D1 and D2 fringes cancel exactly** — this is mathematically guaranteed by quantum mechanics (the no-signaling theorem).

4. **No information travels backward in time** — Bob's choice of measurement on the idler cannot affect what Alice sees at D0.

5. **The "delayed choice" is real but not retrocausal signaling** — the correlations ARE there, but they can only be extracted with classical communication.

---

**Next**: [02_no_signaling_demonstration.ipynb](02_no_signaling_demonstration.ipynb) — A rigorous mathematical proof that no-signaling holds.